# Class Workbook

## In class activity

In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib.pyplot import subplots
#import statsmodels.api as sm
from plotnine import *
import plotly.express as px
import statsmodels.formula.api as sm
#import ISLP as islp

import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression

import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

### Ames Housing data


Please take a look at the Ames Hoursing data.

In [2]:
ames_raw=pd.read_csv("ames_raw.csv")

Use data of `ames_raw` up to 2008 predict the housing price for the later years.

In [3]:
ames_raw_2009, ames_raw_2008= ames_raw.query('`Yr Sold`>=2008').copy(), ames_raw.query('`Yr Sold` <2008').copy()

In [6]:
ames_raw_2008

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
1611,1612,526352080,20,RL,85.0,10667,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,167300
1612,1613,526352090,20,RL,85.0,10628,Pave,NaN,Reg,Lvl,...,0,NaN,GdWo,NaN,0,4,2007,WD,Normal,167000
1613,1614,526355170,20,RL,95.0,13651,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,2,2007,WD,Normal,244000
1614,1615,526355190,20,RL,95.0,15865,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,10,2007,WD,Normal,268000
1615,1616,527105140,60,RL,NaN,12394,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,10,2007,WD,Family,225000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


Use the following loss function calculator.

In [4]:
def calc_loss(prediction,actual):
  difpred = actual-prediction
  RMSE =pow(difpred.pow(2).mean(),1/2)
  operation_loss=abs(sum(difpred[difpred<0]))+sum(0.1*actual[difpred>0])
  return RMSE,operation_loss

Use a simple neural network model.

In [45]:
# Data Loader
class DataLoader:
    def __init__(self, data, target, batch_size=32):
        # Assume data and target are already tensors or numpy arrays
        self.data = torch.tensor(data, dtype=torch.float32)
        self.target = torch.tensor(target, dtype=torch.float32)
        self.batch_size = batch_size
        self.n_samples = self.data.size(0) # Assuming data is 2D (samples, features)
        self.n_batches = math.ceil(self.n_samples / self.batch_size)
    
    def __iter__(self):
        self.current_batch = 0 # Reset counter each time iter is called
        return self

    def __next__(self):
        if self.current_batch >= self.n_batches:
            raise StopIteration
        
        start = self.current_batch * self.batch_size
        end = start + self.batch_size
        if end > self.n_samples:
            end = self.n_samples
            
        self.current_batch += 1
        return self.data[start:end], self.target[start:end]

    def __len__(self):
        return self.n_batches

In [195]:
from sklearn.preprocessing import StandardScaler
X_train = ames_raw_2008[['Lot Area', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd',
       'Fireplaces', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch',
       '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold',
       'Yr Sold']]
y_train = ames_raw_2008['SalePrice']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

X_test = ames_raw_2009[['Lot Area', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd',
       'Fireplaces', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch',
       '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold',
       'Yr Sold']]
y_test = ames_raw_2009['SalePrice']
X_test_scaled = scaler.transform(X_test)

In [197]:
amesloader = DataLoader(data = X_train_scaled, target = y_train.values)

In [198]:
# Module Class

class NeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size = 4, output_size = 1):
        super(NeuralNetwork, self).__init__()
        self.Linear1 = nn.Linear(input_size, hidden_size)  
        # output of : matrix 1 x input_size * input_size x hidden_size = 1 x hidden_size
        self.Linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.Linear1(x)
        out = self.Linear2(out)
        return out

In [199]:
torch.manual_seed(42);
NN = NeuralNetwork(input_size = X_train.shape[1])
print(NN.parameters)

<bound method Module.parameters of NeuralNetwork(
  (Linear1): Linear(in_features=24, out_features=4, bias=True)
  (Linear2): Linear(in_features=4, out_features=1, bias=True)
)>


In [200]:
# Define loss Function
loss_function = nn.MSELoss()

# Optimizer
optimizer =  torch.optim.RMSprop(NN.parameters(), lr = 0.001)

# set iteration
epochs = 10

In [201]:
# Fit the model
for epoch in range(epochs):
    total_loss = 0
    for X_batch, y_batch in amesloader:
               
        # Forward pass
        outputs = NN(X_batch)
        # Compute Loss
        loss = loss_function(outputs, y_batch.view(-1, 1))
        #prediction = NN(X_batch)
        pred = outputs.data
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(amesloader)
    print(f'Epoch {epoch+1}, Average Loss: {round(avg_loss,0)}')

Epoch 1, Average Loss: 40096547596.0
Epoch 2, Average Loss: 40103195258.0
Epoch 3, Average Loss: 40474898871.0
Epoch 4, Average Loss: 40390183595.0
Epoch 5, Average Loss: 40407063357.0
Epoch 6, Average Loss: 40284007473.0
Epoch 7, Average Loss: 40592889076.0
Epoch 8, Average Loss: 40521839567.0
Epoch 9, Average Loss: 40693621419.0
Epoch 10, Average Loss: 41117058974.0


When you decide on your model use the following to come up with your test loss.

In [202]:
NN.eval()  # set model to evaluation mode

testloader = DataLoader(data = X_test_scaled, target = y_test.values,  batch_size = X_test.shape[0])

with torch.no_grad():
    for X, Y in testloader:

        outputs = NN(X)
        # get the predictions
        predicted = outputs.squeeze()
        predicted = predicted.data.numpy()
        # update results
        # print(Y.view(-1, 1).squeeze())
    print(calc_loss(np.array(predicted), Y.view(-1, 1).squeeze()))
    
predictions_tensor = torch.tensor(predicted)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# If you're using a PyTorch loss function (like MSELoss for regression tasks):
loss_function = torch.nn.MSELoss()
loss = loss_function(predictions_tensor, y_test_tensor)

print(loss.item())


(tensor(194661.5156), tensor(28764556.))
37894414336.0


Try to answer the following additional questions.

- Does your model indicate a good fit?


- How does your model result compare to the previous models you fit?


- Can you explain what feature was important determinant of the price?

The loss that I got from the linear regression using the same features is:

33892139.86634926

Compare it with the loss of neural network, 28764556.0.


The NN model greatly reduce the loss.

### COVID 19 Survival in Mexico

Let's revisit COVID-19 in Mexico dataset from the [Mexican government](https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico).  This data is a version downloaded from [Kaggle](https://www.kaggle.com/datasets/meirnizri/covid19-dataset?resource=download).  The raw dataset consists of 21 unique features and 1,048,576 unique patients. In the Boolean features, 1 means "yes" and 2 means "no". values as 97 and 99 are missing data.

- sex: 1 for female and 2 for male.
- age: of the patient.
- classification: COVID test findings. Values 1-3 mean that the patient was diagnosed with COVID in different degrees. 4 or higher means that the patient is not a carrier of COVID or that the test is inconclusive.
- patient type: type of care the patient received in the unit. 1 for returned home and 2 for hospitalization.
- pneumonia: whether the patient already have air sacs inflammation or not.
- pregnancy: whether the patient is pregnant or not.
- diabetes: whether the patient has diabetes or not.
- copd: Indicates whether the patient has Chronic obstructive pulmonary disease or not.
- asthma: whether the patient has asthma or not.
- inmsupr: whether the patient is immunosuppressed or not.
- hypertension: whether the patient has hypertension or not.
- cardiovascular: whether the patient has heart or blood vessels related disease.
- renal chronic: whether the patient has chronic renal disease or not.
- other disease: whether the patient has other disease or not.
- obesity: whether the patient is obese or not.
- tobacco: whether the patient is a tobacco user.
- usmr: Indicates whether the patient treated medical units of the first, second or third level.
- medical unit: type of institution of the National Health System that provided the care.
- intubed: whether the patient was connected to the ventilator.
- icu: Indicates whether the patient had been admitted to an Intensive Care Unit.
- date died: If the patient died indicate the date of death, and 9999-99-99 otherwise.

In [ ]:
import zipfile
Train_COVID= pd.read_csv('Train_COVID.zip',compression='zip')
Test_COVID= pd.read_csv('Test_COVID.zip',compression='zip')

- Fit a sequence model that predicts the number of cases a week a head.

- Modify your model to make prediction for different gender.


Your code:

Your answer:

~~~
Please write your answer in full sentences.


~~~

## Problem set

### Writing your own gradient decent

Consider the simple function $R(\beta) = sin(\beta) + \beta/10$.

(a) Draw a graph of this function over the range $\beta \in [−6, 6]$.
Your code:

In [ ]:
#
#

(b) What is the derivative of this function?

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(c) Given $\beta_0 = 2.3$, run gradient descent to find a local minimum of $R(\beat)$ using a learning rate of $\rho= 0.1$. Show each of $\beta_0,\beta_1,\dots$ in your plot, as well as the final answer.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(d) Repeat with $\beta_0 = 1.4$.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### Default

Fit a neural network to the Default data. Use a single hidden layer with 10 units, and dropout regularization. Have a look at Labs 10.9.1–10.9.2 for guidance. Compare the classification performance of your model with that of linear logistic regression.

Your code:

In [234]:
import ISLP
default = ISLP.load_data('Default')

In [235]:
default['student'] = default['student'].apply(lambda x: 1 if x == 'Yes' else 0)
default['default'] = default['default'].apply(lambda x: 1 if x == 'Yes' else 0)

In [236]:
print(default.default.mean())

default.isna().sum()

0.0333


default    0
student    0
balance    0
income     0
dtype: int64

In [308]:
from torch.utils.data import Dataset, DataLoader
# Data Loader
class defaultDataset(Dataset):
    def __init__(self, data, target):
        # Assume data and target are already tensors or numpy arrays
        self.data = torch.tensor(data, dtype=torch.float32)
        self.target = torch.tensor(target, dtype=torch.float32)
        # self.batch_size = batch_size
        self.n_samples = self.data.shape[0]
        
    # def __iter__(self):
    #     self.current_batch = 0
    #     return self
    
    # def __next__(self):
    #     if self.current_batch >= math.ceil(self.n_samples / self.batch_size):
    #         raise StopIteration
        
    #     start = self.current_batch * self.batch_size
    #     end = start + self.batch_size
    #     if end > self.n_samples:
    #         end = self.n_samples
            
    #     self.current_batch += 1
        # return self.data[start:end], self.target[start:end]

    def __len__(self) :
        return self.n_samples
    
    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]
    
# module class    
class NeuralNetworkClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size = 4, output_size = 1):
        super(NeuralNetworkClassifier, self).__init__()
        self.Linear1 = nn.Linear(input_size, output_size)
        # self.Linear2 = nn.Linear(hidden_size, output_size)
        # self.Relu = nn.ReLU()
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.Linear1(x)
        out = self.Sigmoid(out)
        return out


In [309]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(default[['student', 'balance', 'income']], default['default'], test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [310]:

defTrain = defaultDataset(data =X_train_scaled, target = y_train.values)
trainLoader = DataLoader(defTrain, batch_size = 200)
# straitifiedDefLoader = StratifiedBatchLoader(data = X_train_scaled, target = y_train.values, batch_size = 200)

NNclf  = NeuralNetworkClassifier(input_size = X_train_scaled.shape[1],  hidden_size=4, output_size=1)
print(NNclf.parameters)

<bound method Module.parameters of NeuralNetworkClassifier(
  (Linear1): Linear(in_features=3, out_features=1, bias=True)
  (Sigmoid): Sigmoid()
)>


In [311]:
# Define loss Function
loss_function = nn.BCELoss()

# Set the optimzer as Stochastic Gradient Descent with a learning rate of 0.01
optimizer = torch.optim.SGD(NNclf.parameters(), lr=0.01)

epochs = 10

Numerical Stability in Loss Function: The Binary Cross Entropy (BCE) loss can suffer from numerical instability when the predicted probabilities are very close to 0 or 1. PyTorch provides BCEWithLogitsLoss, which combines a Sigmoid layer and the BCELoss in one single class. This is more numerically stable than using a plain Sigmoid followed by a BCELoss. Since your model ends with a Sigmoid activation, consider removing this last Sigmoid and using BCEWithLogitsLoss instead.

In [312]:
correct, total = 0, 0

# Define empty list variables that store the losses and accuracies at each epoch
losses = []
accuracies = []

for epoch in range(epochs):
    total = 0
    correct = 0
    for inputs, targets in trainLoader:
        outputs = NNclf(inputs)
        loss = loss_function(outputs, targets.view(-1,1))

        predicted = (outputs > 0.5).float()  # Ensure outputs are converted to 0 or 1.
        
        total += targets.size(0)
        correct += (predicted == targets.view(-1,1)).int().sum().item()  # Correctly sum boolean comparisons.

        optimizer.zero_grad() 
        loss.backward() 
        # Apply gradient clipping before the optimizer step
        torch.nn.utils.clip_grad_norm_(NNclf.parameters(), max_norm=1.0)
        
        optimizer.step()
    
        # print(correct)
        # print(total)

    acc = correct / total
    losses.append(loss.item())
    accuracies.append(acc)
    print(f"epoch {epoch}  loss: {loss:.5f}  accuracy: {acc:.5f}")

epoch 0  loss: 0.86541  accuracy: 0.05850
epoch 1  loss: 0.75372  accuracy: 0.18762
epoch 2  loss: 0.66189  accuracy: 0.49512
epoch 3  loss: 0.58651  accuracy: 0.80588
epoch 4  loss: 0.52452  accuracy: 0.95388
epoch 5  loss: 0.47330  accuracy: 0.96700
epoch 6  loss: 0.43072  accuracy: 0.96700
epoch 7  loss: 0.39507  accuracy: 0.96700
epoch 8  loss: 0.36499  accuracy: 0.96700
epoch 9  loss: 0.33941  accuracy: 0.96700


In [ ]:
NN.eval()  # set model to evaluation mode
default_test = defaultDataset(data = X_test_scaled, target = y_test.values)
testloader = DataLoader(default_test,  batch_size = X_test.shape[0])

Your answer:

~~~
Please write your answer in full sentences.


~~~

### IMDb

Repeat the analysis of Lab 10.9.5 on the IMDb data using a similarly structured neural network. We used 16 hidden units at each of two hidden layers. Explore the effect of increasing this to 32 and 64 units per layer, with and without 30% dropout regularization.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### NYSE

Fit a lag-5 autoregressive model to the NYSE data, as described in the text and Lab 10.9.6. Refit the model with a 12-level factor representing the month. Does this factor improve the performance of the model?


Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### NYSE 2
In Section 10.9.6, we showed how to fit a linear AR model to the
NYSE data using the `LinearRegression()` function. However, we also
mentioned that we can “flatten” the short sequences produced for
the RNN model in order to fit a linear AR model. Use this latter
approach to fit a linear AR model to the NYSE data. Compare the test
R2 of this linear AR model to that of the linear AR model that we fit
in the lab. What are the advantages/disadvantages of each approach?

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

Repeat the previous exercise, but now fit a nonlinear AR model by
“flattening” the short sequences produced for the RNN model.

 Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### NYSE 3

Consider the RNN fit to the NYSE data in Section 10.9.6. Modify the code to allow inclusion of the variable day_of_week, and fit the RNN. Compute the test $R^2$.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

### CNN on photo

From your collection of personal photographs, pick 10 images of animals
(such as dogs, cats, birds, farm animals, etc.). If the subject
does not occupy a reasonable part of the image, then crop the image.
Now use a pretrained image classification CNN as in Lab 10.9.4 to
predict the class of each of your images, and report the probabilities
for the top five predicted classes for each image.

Your code:

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~